In [ ]:
#!git clone https://github.com/shiftmark/HPA-SingleCellClassification.git
#!git clone https://github.com/CellProfiling/HPA-Cell-Segmentation.git
#!cd HPA-Cell-Segmentation && sh install.sh
#!pip uninstall PIL && pip uninstall Pillow -y && pip install Pillow
# restart for Pillow

In [ ]:
import sys
import os
import random
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
sys.path.insert(0, '../src/')
sys.path.insert(0, '../tests/')


import test_download_images_and_save_masks as dg
#test.test_on(num_img=10)
from utils.download_files import DownloadFile
from utils.create_tfrecord import CreateTFRecord
from utils.parse_example import ParseExample
from train.helpers import plot_images, plot_history
from train.augment import Augment
from train.get_model import GetModel

SAVE_IMG_TO = GET_IMG_FROM = './assets/images'
SAVE_TFREC_TO = './assets/tfrec'
SAVE_MASKS_TO = './assets/masks'


In [ ]:
dg.download(8, SAVE_IMG_TO)

In [ ]:
img_names = sorted(os.listdir(f'{GET_IMG_FROM}/tc'))
labels = dict()

for img_name in img_names:
    dummy_label = random.randint(0,18)
    name = img_name.split('.')[0]
    labels[name] = dummy_label


In [ ]:
img_names

In [ ]:
CreateTFRecord(f'{GET_IMG_FROM}/tc', labels).write_to(SAVE_TFREC_TO)

In [ ]:
imgs = []#'/content/sample_data/hpa/images/921_B9_1_blue.png']

dataset = tf.data.TFRecordDataset(f'{SAVE_TFREC_TO}/record.tfrec')

for i in dataset.take(10):
    #print(i.shape)
    imgs.append(ParseExample(i, return_img_name=True).parse_eg()[1])
    #imgs.append(tf.squeeze(ParseExample(i, return_img_name=False).parse_eg()[0]).numpy())
data = tf.data.Dataset.from_tensor_slices(imgs)
data2 = Augment(flip_horizontal=False,
                seed=tf.random.uniform((2,),maxval=9, dtype=tf.int32),
                contrast={'lower':.1, 'upper':.9},
                brightness={'max_delta':.1},
                saturation={'lower':.2, 'upper':.7}
                ).apply_on(data)

In [ ]:
imgss = []
for i in data2.take(3):
    imgss.append(i.numpy())
plot_images(imgss, show_axes=False)

In [ ]:
imgsss = []
for i in data.take(3):
    imgsss.append(i.numpy())
plot_images(imgsss, show_axes=False)

In [ ]:
model = GetModel().set_backbone("EfficientNetB0", input_shape=(224,224,3), include_top=True)
model = tf.keras.Model(inputs=model.inputs, outputs=model.outputs)
#model.summary()


model2 = GetModel().set_backbone("EfficientNetB7", input_shape=(600,600,3), include_top=False).add_top(23,.4,33)

#model = Unet((32, 800, 3), 7).unet()

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
plot_model(model)

In [10]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [7]:
import datetime
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])


11493376/11490434 [==============================] - 6s 0us/step


In [8]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])


Epoch 1/5
1875/1875 [==============================] - 42s 22ms/step - loss: 0.3582 - accuracy: 0.8928 - val_loss: 0.1064 - val_accuracy: 0.9670
Epoch 2/5
1875/1875 [==============================] - 43s 23ms/step - loss: 0.1020 - accuracy: 0.9686 - val_loss: 0.0913 - val_accuracy: 0.9703
Epoch 3/5
1875/1875 [==============================] - 44s 23ms/step - loss: 0.0681 - accuracy: 0.9795 - val_loss: 0.0814 - val_accuracy: 0.9740
Epoch 4/5
1875/1875 [==============================] - 42s 22ms/step - loss: 0.0510 - accuracy: 0.9836 - val_loss: 0.0721 - val_accuracy: 0.9770
Epoch 5/5
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0407 - accuracy: 0.9866 - val_loss: 0.0624 - val_accuracy: 0.9812


In [9]:
%tensorboard --logdir logs/fit

In [11]:
tf.debugging.experimental.enable_dump_debug_info(log_dir, tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

INFO:tensorflow:Enabled dumping callback in thread MainThread (dump root: logs/fit/20210523-005847, tensor debug mode: FULL_HEALTH)
